second version of training that uses aggregation mlp

In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, NodePiece
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive.tsv"
TRAIN_PATH = "MSCallGraph_0_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_0_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_0_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                     eager = True,
                     create_inverse_triples = True
                     )


In [5]:
model_name = 'nodepiece_0_transductive_2'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200
num_tokens = 20

In [8]:
model = NodePiece(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim,
        aggregation = 'mlp'
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

sampling:   0%|          | 0.00/7.39k [00:00<?, ?it/s]

No symbolic computation of output shape.


Number of parameters: 243200
Space occupied: 972800 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory nodepiece_0_transductive_2 created successfully!


In [10]:
learning_rate = 1e-3
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 3.7206536531448364
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.005259472387822627
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.010096666970206947
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.007678069679014787
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0016327445628121495
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0038409144617617126
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0027368292212486267
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0012991754014105648
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0032271995538847515
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.002263187477647658
Metric: validation.head.optimistic.hits_at_1 = 0.0023346303501945525
Metric: validation.tail.optimistic.hits_at_1 = 0.005447470817120622
Metric: validation.both.optimistic.hits_at_

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 1.7528334856033325
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.006240932459531757
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.02511760061269083
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.01567926653611129
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.002099476056173444
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.011023487895727158
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.006561481859534979
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0016699681383603587
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.008774466406520232
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.005222217272440296
Metric: validation.head.optimistic.hits_at_1 = 0.0019455252918287938
Metric: validation.tail.optimistic.hits_at_1 = 0.005447470817120622
Metric: validation.both.optimistic.hits_at_1 = 0.

...

Step: 86
Metric: loss = 0.3448511064052582
Step: 86
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.6060952653461322
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0631388695655352
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.3346170674558337
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.007454439532011747
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.015163573436439036
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.011309006251394749
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0054488081470940196
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.011694059267896786
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.008571433707495402
Metric: validation.head.optimistic.hits_at_1 = 0.5968871595330739
Metric: validation.tail.optimistic.hits_at_1 = 0.050972762645914396
Metric: validation.both.optimistic.hits_at_1 = 0.32392

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 87
Metric: loss = 0.34482714533805847
Step: 87
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.6047471586646471
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.06570939969470156
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.3352282791796744
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.008304408751428127
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.013731513172388075
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.011017960496246815
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.006191860899990578
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.010695294389474272
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.008443577644732424
Metric: validation.head.optimistic.hits_at_1 = 0.5945525291828794
Metric: validation.tail.optimistic.hits_at_1 = 0.05525291828793774
Metric: validation.both.optimistic.hits_at_1 = 0.32490

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 88
Metric: loss = 0.3497449606657028
Step: 88
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.6051864452770498
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.06584232318204468
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.3355143842295472
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.008498582988977432
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.012926461175084114
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.010712521150708199
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.006253196372592601
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.009994462339236936
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.008123829355914769
Metric: validation.head.optimistic.hits_at_1 = 0.5933852140077821
Metric: validation.tail.optimistic.hits_at_1 = 0.05680933852140078
Metric: validation.both.optimistic.hits_at_1 = 0.325097

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
           0
0   3.720654
1   1.752833
2   1.170424
3   0.888958
4   0.761247
..       ...
83  0.350689
84  0.345576
85  0.344851
86  0.344827
87  0.349745

[88 rows x 1 columns]


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/10.4k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.714852   0.005110     0.003936
hits_at_1                     0.707639   0.000193     0.000193
hits_at_3                     0.713804   0.002890     0.002890
hits_at_5                     0.715634   0.004335     0.004335
hits_at_10                    0.730662   0.007995     0.006647
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.099173   0.027338     0.020089
hits_at_1                     0.079087   0.004335     0.004335
hits_at_3                     0.082458   0.029092     0.006454
hits_at_5                     0.105385   0.030633     0.030440
hits_at_10                    0.116944   0.061073     0.035546
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.407013   0.016224     0.012013
hits_at_1                     0.393363   0.002264     0.002264
hits_at_3                     0.398131  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/2.57k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.601312   0.006973     0.005322
hits_at_1                     0.592218   0.000000     0.000000
hits_at_3                     0.599611   0.002724     0.002724
hits_at_5                     0.606226   0.007004     0.006615
hits_at_10                    0.617510   0.015175     0.009728
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.094897   0.028977     0.022281
hits_at_1                     0.074319   0.006226     0.006226
hits_at_3                     0.082101   0.031907     0.010895
hits_at_5                     0.103113   0.034630     0.034241
hits_at_10                    0.117121   0.061479     0.043191
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.348104   0.017975     0.013802
hits_at_1                     0.333268   0.003113     0.003113
hits_at_3                     0.340856  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/3.27k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.625330   0.006786     0.005243
hits_at_1                     0.616208   0.000306     0.000306
hits_at_3                     0.623853   0.002446     0.002446
hits_at_5                     0.629664   0.005810     0.005810
hits_at_10                    0.642508   0.012844     0.009786
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.095767   0.031017     0.023009
hits_at_1                     0.074312   0.005505     0.005505
hits_at_3                     0.081040   0.035780     0.007645
hits_at_5                     0.104893   0.038532     0.037615
hits_at_10                    0.121101   0.070031     0.047401
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.360549   0.018901     0.014126
hits_at_1                     0.345260   0.002905     0.002905
hits_at_3                     0.352446  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['num_tokens'] = num_tokens
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience
infodict['aggregation'] = 'mlp'

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                      num_tokens   
17                   embedding_dim   
18                   learning_rate   
19                       optimizer   
20                      num_epochs   
21                        patience   
22                     aggregation   

                                                value  
0                              

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)